# Linear Classifiers

## Fitting a line to data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Supose we are measuring how a mouse weight (X) can be used to predict the mouse size(Y). The values are the following:

In [ ]:
X = np.array([0.60316714, 5.13981077, 0.57754654, 3.35880456, 5.28171939,
        9.41578636, 2.43742198, 5.99075038, 2.49605785, 6.83781763,
        0.16296473, 9.29969598])
Y = np.array([15.15613261, 23.89223832, 15.72151754, 16.35859565, 22.06175073,
        27.36346235, 20.4802553 , 24.54353801, 21.22924112, 21.77229456,
        14.94636364, 30.70479942])
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
plt.show()

We train to find a good line that fit the data.

In [ ]:
def draw_line(line, color, ax, values):
    min_value = np.min(values)
    max_value = np.max(values)
    # Generate x-values
    x = np.linspace(min_value, max_value, 100)  # range of x-values
    y = line(x)
    ax.plot(x, y, color=color)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
line = np.poly1d([1.2, 16])
draw_line(line, 'red', ax, X)
plt.show()

Now it comes the question, is it a good one? Is it the best posible line to fit the data?

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
draw_line(np.poly1d([1.2, 16]), 'red', ax, X)
draw_line(np.poly1d([1.4, 14]), 'gray', ax, X)
draw_line(np.poly1d([0.8, 16]), 'cyan', ax, X)
draw_line(np.poly1d([0, 22]), 'green', ax, X)
plt.show()

We can measure now how well the line fits the data by seen how close is it to the data points

In [ ]:
distances = [abs(line(x) - y) for x, y in zip(X, Y)]
print(distances)
print(sum(distances))

Since we want to penalize larger divergences, we square the terms (additionally, the *abs* have some nasty mathematical properties).

In [ ]:
distances = [(line(x) - y)**2 for x, y in zip(X, Y)]

if we add all the distances, the result is named **sum of squared residuals (SSR)**, because the **residuals** are the differences between the real and estimated values.

In [ ]:
sum(distances)

Now lets create a function for performing the evaluation

In [ ]:
def sum_sq_res(line, X, Y):
    return sum((line(x) - y)**2 for x, y in zip(X, Y))

sum_sq_res(line, X, Y)

Lets evaluate the functions we used before

In [ ]:
all_lines = [
    (np.poly1d([1.2, 16]), 'red'), 
    (np.poly1d([1.4, 14]), 'gray'),
    (np.poly1d([0.8, 16]), 'cyan'),
    (np.poly1d([0, 22]), 'green'),
]

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
for l, color in all_lines:
    draw_line(l, color, ax, X)
    print(color, sum_sq_res(l, X, Y))
plt.show()

As you can see, the better the line fit the data, the smaller value it have on the SSR. 

Lets try to find the line with the minimal value. This method is called **Least Squares**. We need to find two values:
- The curve slope, that controls the angle with respect to the horizontal axis
- The curve intercept, that controls the point where the curve cuts the vertical axis.

Consider the horizontal line with the average _y_ value. This is not a good one, but since it is based on data, will be our starting point.



In [ ]:
b = np.average(Y)
print(b)
line = np.poly1d([0, b])

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
draw_line(line, 'red', ax, X)

Now we explore the influence of the slope in the SSR

In [ ]:
for slope in np.arange(-4, 4, 0.1):
    line = np.poly1d([slope, b])
    ssr = sum_sq_res(line, X, Y)
    print(slope, ssr)

In [ ]:
# using a comprehension
points = [(slope, sum_sq_res(np.poly1d([slope, b]), X, Y)) for slope in np.arange(-5, 5, 0.1)]
ssrs = np.array(points)
plt.plot(ssrs[:, 0],ssrs[:, 1])
plt.show()


The ssr function has a derivative that is easy to calculate, allowing us to directly locate the minimum by finding the point where the derivative equals zero.

For this example, since we have a computer, we can directly get the slope with lowest ssr value:

In [ ]:
min_row_index = np.argmin(ssrs[:, 1])
min_slope = ssrs[min_row_index, 0]
min_slope

In [ ]:
b = np.average(Y)
print(b)
line = np.poly1d([min_slope, b])

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
draw_line(line, 'red', ax, X)

We can do a similar operation in both parameters simultaneously to find the best line.

In [ ]:
points = [(slope, c, sum_sq_res(np.poly1d([slope, c]), X, Y)) 
          for slope in np.arange(-5, 5, 0.1) 
          for c in np.arange(10, 20, 0.1)]

In [ ]:
points = np.array(points)
points

Lets plot the SSR values with respect to slope and c

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the points in 3D
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2])

# Set labels and title
ax.set_xlabel('slope')
ax.set_ylabel('c')
ax.set_zlabel('SSV')
ax.set_title('3D Scatter Plot')

# Show the plot
plt.show()


And get the lowest value

In [ ]:
min_row_index = np.argmin(points[:,2])
min_slope, min_c, ssr = points[min_row_index]
min_slope, min_c, ssr

In [ ]:
line = np.poly1d([min_slope, min_c])

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y, color='blue', label='Random Points')
draw_line(line, 'red', ax, X)

Summarizing:
- To fit the model to the available data we need to minimize the sum of squared residual from the model and the data
- To do this, if we know a good candidate parameter interval, we can explore it. 
    - We can also analytically calculate the partial derivatives and find the point where it is zero
- The point where the SSR is minimal can be used in the model. derivative is zero is the one that minimizes the SSR, so it must be used for the final model.

## Evaluating the quality of fitting, the R^2

Lets start by taking only the mouse size, and plotting the line of its average value.

In [ ]:
mean_size = np.mean(Y)

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(np.full(len(Y), 4), Y, color='blue', label='Random Points')
draw_line(np.poly1d([0, mean_size]), 'red', ax, X)

Now, calculate the SSR around the mean

In [ ]:
ssr_mean = sum((mean_size - y)**2 for y in Y)
ssr_mean

The variance around the mean is calculated dividing ssr_mean by the number of elements

In [ ]:
Var_mean = ssr_mean / len(Y)
Var_mean

Now calculate the SSR for our best estimation of the line that relates mouse weight and size

In [ ]:
line = np.poly1d([min_slope, min_c])
ssr_fit = sum((line(x) - y)**2 for x, y in zip(X, Y))
ssr_fit

And its variance

In [ ]:
Var_fit = ssr_fit / len(Y)
Var_fit

We can see that a part of the variation of the the Y variable can be "explained" by taking the X variable into account.
- The variation in the mouse size can be explained partially by taking the mouse weight into cosideration.

R^2 tells us how much of the variation in the mouse size can be explained by taking the mouse weight into acount. The equation is the following:

R^2 = (Var_mean - Var_fit) / Var_mean.

Since the number of elements divide both variances, we can simplify by:

R^2 = (ssr_mean - ssr_fit) / ssr_mean

In [ ]:
R_2 = (ssr_mean - ssr_fit) / ssr_mean
R_2

This means that almost 85% of the variation in mouse size can be explained by taking weight into account using the calculated model. This is quite good indeed!!

Lets see another example

In [ ]:
Y2 = line(X)

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X, Y2, color='blue', label='Random Points')
draw_line(line, 'red', ax, X)

In [ ]:
ssr_mean = sum((mean_size - y)**2 for y in Y2)
ssr_fit = sum((line(x) - y)**2 for x, y in zip(X, Y2))
R_2 = (ssr_mean - ssr_fit) / ssr_mean
R_2

We can see that 100% of the variance in the Y variable can be explained by the X variable using the line equation. Perfect match!

Lets see a final example

In [ ]:
X3 = np.random.randint(1, 10, size=15)
Y3 = np.random.randint(1, 10, size=15)
line3 = np.poly1d([0, np.average(Y3)])

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X3, Y3, color='blue', label='Random Points')
draw_line(line3, 'red', ax, X)

Lets calculate R^2

In [ ]:
mean_size = np.average(Y3)
ssr_mean = sum((mean_size - y)**2 for y in Y3)
ssr_fit = sum((line3(x) - y)**2 for x, y in zip(X3, Y3))
R_2 = (ssr_mean - ssr_fit) / ssr_mean
R_2

In this case, since R^2 is zero, knowing the mouse weight do not helps us to explain the mouse size, and the model we built is useless.

**Note**. We apply this analysis to lines, but it can be perfomed to any other model. 

## Linear classifiers in Scikit-learn

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

data = pd.read_csv('data/heart.csv')
label_encoder = LabelEncoder()
data["Sex"] = label_encoder.fit_transform(data["Sex"])
data["ChestPainType"] = label_encoder.fit_transform(data["ChestPainType"])
data["RestingECG"] = label_encoder.fit_transform(data["RestingECG"])
data["ExerciseAngina"] = label_encoder.fit_transform(data["ExerciseAngina"])
data["ST_Slope"] = label_encoder.fit_transform(data["ST_Slope"])
data.head()

### Simple linear regression models
Lets try to predict the person Age, based on the other parameters

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

X = data.iloc[:, 1:]
y = data.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)


# Print the coefficients
print("Coefficients:", regressor.coef_)
print("Intercept:", regressor.intercept_)


And evaluate it.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

# Calculate predicted values
y_pred = regressor.predict(X)

# Calculate sum of squared errors (SSE)
sse = mean_squared_error(y, y_pred) * len(y)

# Calculate total sum of squares (SST)
sst = np.sum((y - np.mean(y))**2)

# Calculate R-squared value
r_squared = 1 - (sse / sst)

# Calculate p-values using statsmodels
X_t = sm.add_constant(X)  # Add constant column for intercept
model = sm.OLS(y, X_t)
results = model.fit()
p_values = results.pvalues

# Print results
print("Sum of Squared Errors (SSE):", sse)
print("R-squared:", r_squared)
print("P-values:")
for k, p in p_values.items():
    print(k, str(round(p, 2)))

### Logistic classifier
A logistic classifier, also known as logistic regression, is a popular classification algorithm used in machine learning. 
- It is primarily used for binary classification tasks, where the goal is to predict the probability of an input belonging to a particular class.
- The logistic classifier utilizes the logistic function, also called the sigmoid function, to model the relationship between the input features and the target class probabilities. 
- The sigmoid function maps any real-valued input to a value between 0 and 1, representing the probability of the input belonging to the positive class.

In this example, we have the probability of mouse to be obese based on its weight.

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

weights = [1,2, 2.5, 3, 4, 5, 4, 5, 6, 8, 9, 10]
obese_prob = [ 0,0,0,0,0, 0,0,1,1,1,1, 1]

X = np.array([[x] for x in weights])
y = np.array(obese_prob)

Now, lets train a Logistic regression model.

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


# Initialize and train the logistic regression classifier
clf = LogisticRegression()
clf.fit(X, y)

# Plot the decision boundary
x_min, x_max = X.min() - 1, X.max() + 1
xx = np.linspace(x_min, x_max, 100).reshape(-1, 1)
y_pred = clf.predict_proba(xx)[:, 1]

plt.plot(xx, y_pred)
plt.scatter(X, y, c='red')
plt.ylabel('Obese probability')
plt.xlabel('Mouse weight')
plt.title('Logistic Regression')
plt.show()


- The learned model reveals that the probability of obesity tends to approach zero when the weight is closer to 2 or less. 
- Conversely, the probability of obesity tends to approach one when the weight is closer to 8. 
- It is important to note that the resulting probabilities range from 0 to 1, even though the training data only contains zeros and ones.

Now, lets try this model with real data:

In [ ]:
data.head(3)

In [ ]:
data_new = data.copy()
data_new['isOld'] = data.Age > 54
data_new = data_new.drop(['Age'], axis=1)

In [ ]:
X = data_new.drop(['isOld'], axis=1)
y = data_new['isOld']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

Now we evaluate the model

In [ ]:
y_pred = clf.predict_proba(X_test)

This is, for the first 10 objects, the probability of classification in class 0 and 1, or 'young' and 'old'

In [ ]:
np.hstack([y_pred, y_test.to_frame()])[:10, :]

Lets evaluate with accuracy, which assign the class to the highest probability

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate predicted values
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)